In [1]:
%cd ../../

%load_ext autoreload
%autoreload 2

/home/hoanghu/projects/Food-Waste-Optimization


In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [3]:
import plotly.io as pio

pio.templates.default = "seaborn"

# Load data

In [5]:
path_dir_processed = Path("experiments_hoangle/processed")

path_fact = path_dir_processed / "fact.csv"

raw_fact = pd.read_csv(path_fact, header=0, parse_dates=['date'])

raw_fact.head()

,date,restaurant,num_fish,num_chicken,num_vegetarian,num_meat,num_NotMapped,num_vegan,num_customer_in,num_customer_out,num_rcpts,amnt_waste_customer,amnt_waste_coffee,amnt_waste_kitchen,amnt_waste_hall
0,2023-01-02,Chemicum,85.0,0.0,0.0,171.0,1.0,91.0,NaN,NaN,272.0,4.70,1.2,12.0,0.0
1,2023-01-03,Chemicum,163.0,0.0,32.0,78.0,1.0,120.0,NaN,NaN,327.0,5.00,1.4,14.8,0.0
2,2023-01-04,Chemicum,70.0,0.0,0.0,218.0,3.0,137.0,NaN,NaN,351.0,4.15,4.0,7.1,0.0
3,2023-01-05,Chemicum,232.0,85.0,0.0,2.0,4.0,178.0,NaN,NaN,437.0,10.00,3.3,8.5,0.0
4,2023-01-06,Chemicum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Explore

In [12]:
COLS_WASTE = [
    'amnt_waste_customer',
    'amnt_waste_coffee',
    'amnt_waste_kitchen',
    'amnt_waste_hall'
]

In [132]:
cols = [
    'date',
    'restaurant',
    *COLS_WASTE
]

wastes = raw_fact[cols].dropna(axis=0)

wastes['weekday'] = wastes['date'].dt.weekday

wastes['is_friday'] = wastes['weekday'] == 4
wastes = wastes.groupby(['restaurant', 'is_friday'])[COLS_WASTE].mean()

wastes.head()

amnt_waste_customer  amnt_waste_coffee  \
restaurant is_friday                                           
Chemicum   False                15.861049           2.845468   
           True                 14.322941           3.886029   
Exactum    False                 6.494653           1.654097   
           True                  5.971724           1.607931   
Physicum   False                 0.269698          10.003970   

                      amnt_waste_kitchen  amnt_waste_hall  
restaurant is_friday                                       
Chemicum   False               16.609288         2.286517  
           True                18.844853         3.026471  
Exactum    False                7.924861         0.341250  
           True                 9.662069         1.280000  
Physicum   False                7.148643         0.160754

## Analyze foodwaste by restaurant

### In total

In [79]:
results = (
    wastes
    .mean(axis=1)
    .reset_index()
    .pivot(index='restaurant', columns='is_friday', values=0)
    .rename(columns={False: 'other_weekdays', True: 'friday'})
    .reset_index()
    # .drop(columns=['is_friday'])
)

results

is_friday,restaurant,other_weekdays,friday
0,Chemicum,9.400581,10.020074
1,Exactum,4.103715,4.630431
2,Physicum,4.395766,4.036196


In [81]:
fig = make_subplots(
    rows=1, cols=1,
    specs=[
        [{'type': 'bar'}]
    ]
)

fig.add_trace(
    go.Bar(
        x=results['restaurant'],
        y=results['friday'],
        name='Friday'
    ),
    row=1, col=1
)
fig.add_trace(
    go.Bar(
        x=results['restaurant'],
        y=results['other_weekdays'],
        name='Other weekdays'
    ),
    row=1, col=1
)

fig.update_layout(
    height=600, 
    width=800,
    title_text="<b>Average foodwaste of Friday and other weekdays by restaurant</b>",
    title_font_size=25,
    xaxis={'tickangle': -90, 'tickfont': {'size': 16}},
    legend={'orientation': 'h', 'xanchor': "center", 'y': 1, 'yanchor': "bottom", 'x': 0.5, 'font': {'size': 16}},
    title_x=0.5,
)
fig.show()

### By waste type

In [253]:
RESTAURANTS = ['Chemicum', 'Exactum', 'Physicum']

In [254]:
fig = make_subplots(
    rows=3, cols=1,
    specs=[
        [{'type': 'bar'}], [{'type': 'bar'}], [{'type': 'bar'}]
    ],
    subplot_titles=RESTAURANTS
)

In [255]:
results = (
    wastes
    .reset_index()
    .rename(
        columns={
            'amnt_waste_customer': 'Customer',
            'amnt_waste_coffee': 'Coffee',
            'amnt_waste_kitchen': 'Kitchen',
            'amnt_waste_hall': 'Hall',
        }
    )
)

for i, restaurant in enumerate(RESTAURANTS):
    showlegend = i == 0

    wastes_restaurant = (
        results[results['restaurant'] == restaurant]
        .reset_index()
        .drop(columns=['index', 'restaurant'])
        .T
        .reset_index()
        .iloc[1:]
        .rename(columns={'index': 'Waste type', 0: 'Other weekdays', 1: 'Friday'})
    )

    fig.add_trace(
        go.Bar(
            x=wastes_restaurant['Waste type'],
            y=wastes_restaurant['Friday'],
            name='Friday',
            legendgroup='1',
            showlegend=showlegend,
            marker=dict(color='#4c72b0')
        ),
        row=i+1, col=1
    )
    fig.add_trace(
        go.Bar(
            x=wastes_restaurant['Waste type'],
            y=wastes_restaurant['Other weekdays'],
            name='Other weekdays',
            legendgroup='2',
            showlegend=showlegend,
            marker=dict(color='#dd8452')
        ),
        row=i+1, col=1
    )

In [256]:
fig.update_layout(
    height=800, 
    width=800,
    legend={'orientation': 'h', 'xanchor': "center", 'y': 1.03, 'x': 0.5, 'yanchor': "bottom",'font': {'size': 16}},
    title={'xanchor': "center", "text": "<b>Waste amount by type and restaurant</b>", 'font': {'size': 25}, 'y': 0.98},
)
fig.update_xaxes(tickangle=-40, tickfont={'size': 16})
fig.show()